# Introduction to pandas

This week we will explore `pandas`, a popular data science library for Python. In order to apply linear algebra tools to data, it is useful to be able to actually load and manipulate data sets in Python. One way to achieve this is with the `pandas` library. In order to fully exectue the code in this notebook, make sure you download the datafiles provided on LATTE and put them in the same directory as this Jupyter notebook.

A *library* is a collection of Python code that adds functionality that is not present in the core language. In order to use the features of a library, we must first import it. This is accomplished with an import statement, like the following:

In [1]:
import pandas

Import statements are usually placed at the top of a Python script. Make sure you include any import statements you need in the scripts you submit for assignments.

To access the functionality provided by the library, we prefix commands by the name of the library. Since "pandas" has six whole letters in it, and we are fundamentally lazy people, it is handy to use a shortcut. We can instead use the following command:

In [2]:
import pandas as pd

This provides the functionality of the `pandas` library via the much more concise `pd` shortcut. (But you could use any alias you like instead of `pd`.)

Pandas organizes data into table-like structures called `DataFrame`s. A DataFrame consists of one or more named columns, and one or more indexed rows. Here is an example. (Execute the cell to see a representation of the DataFrame.)

In [3]:
pd.DataFrame({"Name": ["New York", "Los Angeles", "Chicago"], "Population": [8336000, 3979000, 2693000]})

,Name,Population
0,New York,8336000
1,Los Angeles,3979000
2,Chicago,2693000


As you can see, the DataFrame has three rows (indexed 0,1, and 2) and two columns (named "Name" and "Population"). Some of the entries are strings, and some are integers; other types are possible as well.

In order to construct a DataFrame, we gave a dictionary whose keys are the names of the columns of the DataFrame, and whose values are lists with the corresponding entries of those columns. This is a good way to think about a DataFrame, although DataFrames are overall substantially more complicated and feature-rich.

It is also possible to use row indices other than consecutive integers. The row indices are just a way of referring to different rows of the DataFrame, just like the column names refer to different columns. For example, here is another DataFrame with string indices for the rows:

In [ ]:
pd.DataFrame({"Population": [8336000, 3979000, 2693000]}, index=["New York", "Los Angeles", "Chicago"])

In this DataFrame, we have only one column (Population), and instead of integer indices we use city names as indices.

Usually, instead of constructing DataFrames by hand, we will read in datasets stored as files. To do so, we need a data file with tabular data (like an Excel file or a CSV file), and we need to know the file's "path" or location in our filesystem. It is usually easiest to specify paths *relative to your script or Jupyter notebook*. For example, if you put the datafile in the same location as your Jupyter notebook or Python script, then the path is just the filename (including any extension, like ".csv"). If it is in a different location, you can separate filesystem directories using the "/" character. This notebook will assume that all data files are in the same location as the Jupyter notebook.

The following command reads a data file with CO2 emissions (in thousands of tons) of various regions, and stores it as a pandas DataFrame in the variable `data`.

In [4]:
data = pd.read_csv("../Data/co2emissions.csv")

FileNotFoundError: [Errno 2] File ../Data/co2emissions.csv does not exist: '../Data/co2emissions.csv'

Once we have a pandas DataFrame, there are many ways to summarize the DataFrame and understand what it is. For example,
- `data.head()` will display the first 5 rows
- `data.shape` will tell you how many rows and how many columns the DataFrame has
- `data.info()` will list all the columns, along information like their type (e.g. integer or floating point)
- `data.describe()` will give some basic statistics for each of the numerical columns

In [ ]:
data.head()

Notice that many of the entries in this dataset are `NaN`. This is a special floating-point value that means "not a number". In the context of this dataset, it indicates that the corresponding data is missing or does not exist. Such data is also called "null" data.

## Indexing and selecting data

To access a particular column of data from a DataFrame, we use square-bracket notation along with the column name - exactly as if the DataFrame were a dictionary whose keys are the column names.

In [ ]:
data["Country Name"]

Accessing a column in this way produces a pandas `Series`, which is roughly analogous to a Python list. For example, we can access an individual item in the Series by giving its index. (Notice in the following that we first use square-bracket indexing to get the "Country Name" column of the DataFrame, and then use square-bracket indexing again to get a particular entry of the resulting Series.)

In [1]:
data["Country Name"][0]

NameError: name 'data' is not defined

What if, instead, we want to get a single row of the DataFrame rather than a single column? In that case, we can use square-bracket indexing along with the `iloc` attribute of the DataFrame, like this:

In [ ]:
data.iloc[1] # get the second row of the DataFrame

Even if you have complicated indexes for rows (as in the city population example near the start of this notebook), you can still access the rows using consecutive numerical indices starting from 0 using `iloc`. If you want to access a row using more complex indices you have specified, you can use the `loc` attribute instead of `iloc`.

In [3]:
import pandas as pd
cities = pd.DataFrame({"Population": [8336000, 3979000, 2693000]}, index=["New York", "Los Angeles", "Chicago"])
cities.loc["New York"]

Population    8336000
Name: New York, dtype: int64

If you want to use a particular column as the index (so that you can access corresponding rows using square-bracket notation along with `loc`), you can use the `set_index` method. This produces a new DataFrame, so you'll need to store the result in a new variable.

In [ ]:
data_reindexed = data.set_index("Country Name")
data_reindexed.loc["China"]

Regardless of whether you access a single row of a DataFrame using `loc` or `iloc`, or a single column using square-bracket indexing, you end up with a Series. In the case of a Series for a single row (and also in the case of a Series for a single column of a DataFrame with a fancy index), you can access the entries using square-bracket indexing, with two kinds of indices:
1. integers starting from 0 giving the position of the item you want to access in the Series
2. the name of the index (e.g., the column name) as a string

So, for example, the following two `print` statements do the same thing:

In [ ]:
china_row = data_reindexed.loc["China"]
print(china_row[1])
print(china_row["1960"])

## Iterating over DataFrames

You can iterate over a DataFrame using a `for` loop. Can you guess what the individual items will be? If you are thinking of a DataFrame as something like a dictionary whose keys are the column names and whose values are the list of entries in each column, then you will guess the right answer: iterating directly over a DataFrame using a `for` loop gives you the names of each of the columns.

In [ ]:
for column_name in data:
    print(column_name)

It is also useful to be able to iterate over the *rows* of a DataFrame. This is possible using the `iterrows` method. This method gives you both the contents of the row (the values in each column, as a Series) along with the index of the row. 

In [ ]:
for index, row in data_reindexed.iterrows():
    # index is, e.g., Aruba or Afghanistan
    # row is a Series with the values for each column
    ... # do whatever you want with index and row here

## Convenience functions

The combination of iterating over a DataFrame and indexing into it can in principle let you compute whatever you'd like about the data. For example, we can compute the maximum amount of CO2 emissions by any country in the year 2010 as follows:

In [ ]:
largest = 0
for index, row in data.iterrows():
    if row["2010"] > largest:
        largest = row["2010"]
print(largest)

But Pandas also provides an enormous suite of methods for doing computations like this without having to manually iterate over the data. We won't cover these in detail in this course, though you are welcome to research them on your own. However, we will mention a few examples.
- `max()` computes the maximum value
- `mean()` computes the average value
- `count()` gives the total number of values
- `sum()` gives the sum of the values

So, for example, the following code also computes the maximum amount of CO2 emissions by any country in the year 2010.

In [ ]:
data["2010"].max()

## Conditional selection

One very fancy feature of pandas DataFrames is the ability to select rows based on Boolean expressions. For example, suppose we are interested in the subset of the DataFrame corresponding to countries with more than 1 billion tons of CO2 emissions in 2010. We can produce a Series of True/False values indicating which rows belong to this subset by using a Boolean expression as follows:

In [ ]:
data["2010"] > 1000000

The real magic is that we can then use the resulting Boolean Series to select the subset of the DataFrame corresponding to the True values:

In [ ]:
big_emitters = data[data["2010"] > 1000000]
print(big_emitters)